<a href="https://colab.research.google.com/github/ykitaguchi77/GAN/blob/master/one_shot_talking_face.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**One Shot Talking Face**

http://cedro3.com/ai/one-shot-talking-face/


In [ ]:
#@title #**Setup**
!git lfs install
%cd /content
!git clone https://huggingface.co/camenduru/pocketsphinx-20.04-t4 pocketsphinx
%cd  /content/pocketsphinx
!sudo cmake --build build --target install
%cd /content
!git clone https://huggingface.co/camenduru/one-shot-talking-face-20.04-t4 one-shot-talking-face
%cd /content/one-shot-talking-face
!pip install -r /content/one-shot-talking-face/requirements.txt
!chmod 755 /content/one-shot-talking-face/OpenFace/FeatureExtraction
!mkdir /content/out
!apt install -qq libgtk2.0-0 jq -y
!pip install -q imageio-ffmpeg

In [22]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename

In [ ]:
from IPython.display import Image
try:
  filename = take_photo()
  print('Saved to {}'.format(filename))

  # Show the image which was just taken.
  display(Image(filename))
except Exception as err:
  # Errors will be thrown if the user does not have a webcam or if they do not
  # grant the page permission to access it.
  print(str(err))

In [ ]:
# Google Colabでのスクリプト:

!pip install mediapipe opencv-python

import mediapipe as mp
import cv2
from google.colab.patches import cv2_imshow

# MediapipeのFace detectionモデルを初期化
mp_face_detection = mp.solutions.face_detection
mp_drawing = mp.solutions.drawing_utils
face_detection = mp_face_detection.FaceDetection(min_detection_confidence=0.2)

# 画像の読み込み
image = cv2.imread('/content/one-shot-talking-face/photo.jpg')
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# 顔の検出
results = face_detection.process(image_rgb)

# 検出された顔を抜き出して表示
if results.detections:
    for detection in results.detections:
        bboxC = detection.location_data.relative_bounding_box
        ih, iw, _ = image.shape
        x, y, w, h = int(bboxC.xmin * iw), int(bboxC.ymin * ih), int(bboxC.width * iw), int(bboxC.height * ih)

        # 余白を追加
        padding = int(w * 0.3)  # 顔の幅の30%を余白として追加
        x = max(0, x - padding)
        y = max(0, y - padding)
        w = min(iw - x, w + 2 * padding)
        h = w  # 正方形にする

        face_image = image[y:y+h, x:x+w]
        cv2_imshow(face_image)

# 画像の保存
cv2.imwrite('/content/one-shot-talking-face/cropped_face.jpg', face_image)
print("Face image saved to /content/one-shot-talking-face/cropped_face.jpg")


In [ ]:
#@title #**make movie**
import os, random, torchaudio
from IPython.display import HTML
from base64 import b64encode
from IPython.display import clear_output

def show_video(video_path, video_width = 256):
  video_file = open(video_path, "r+b").read()
  video_url = f"data:video/mp4;base64,{b64encode(video_file).decode()}"
  return HTML(f"""<video width={video_width} controls><source src="{video_url}"></video>""")

#@markdown -Select wav_file_name from the one-shot-talking-face/samples/audios folder.\
#@markdown -Select image_file_name from the one-shot-talking-face/samples/imgs folder.

wav_file_name = 'abstract.wav' #@param {type:"string"}
image_file_name = 'KevinBrady0#0.jpg' #@param {type:"string"}

wav_file = "/content/one-shot-talking-face/samples/audios/" + wav_file_name
#image_file = "/content/one-shot-talking-face/samples/imgs/" + image_file_name
image_file = "/content/one-shot-talking-face/cropped_face.jpg"


waveform, sample_rate = torchaudio.load(wav_file)
torchaudio.save(wav_file, waveform, sample_rate, encoding="PCM_S", bits_per_sample=16)

os.environ['wav_file'] = wav_file
os.environ['image_file'] = image_file
random_int = str(random.randint(1, 1000000))

!mkdir /content/train
!cp $wav_file /content/train/audio.wav
!cp $image_file /content/train/image.png

!pocketsphinx -phone_align yes single /content/train/audio.wav $text | jq '[.w[]|{word: (.t | ascii_upcase | sub("<S>"; "sil") | sub("<SIL>"; "sil") | sub("\\(2\\)"; "") | sub("\\(3\\)"; "") | sub("\\(4\\)"; "") | sub("\\[SPEECH\\]"; "SIL") | sub("\\[NOISE\\]"; "SIL")), phones: [.w[]|{ph: .t | sub("\\+SPN\\+"; "SIL") | sub("\\+NSN\\+"; "SIL"), bg: (.b*100)|floor, ed: (.b*100+.d*100)|floor}]}]' > /content/test.json
%cd /content/one-shot-talking-face
!python -B test_script.py --img_path /content/train/image.png --audio_path /content/train/audio.wav --phoneme_path /content/test.json --save_dir /content/train

os.environ['ran_num'] = random_int
!cp /content/train/image_audio.mp4 "/content/out/${ran_num}.mp4"

clear_output()

show_video(f"/content/out/{random_int}.mp4")

In [5]:
#@title #**Download movie** ( for google chrome)
from google.colab import files
file_path =f"/content/out/{random_int}.mp4"
files.download(file_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>